In [ ]:
import geopandas as gp
import csv
import pandas as pd
from shapely.geometry import LineString
import mysql.connector
from shapely.geometry import Polygon, mapping
import datetime
import json 

In [25]:
file = "./PA/pa_shapes/tl_2012_42_vtd10.shp"    

precincts = gp.read_file(file)

In [21]:
#reader = csv.DictReader('../data/NY_county.csv')
reader = csv.reader(open('./PA_county.csv', 'r'))
d = {}
for row in reader:
    #id = row['County FIPS']
    key = row[3]
    value = row[5]
    d[key] = value
    
#print (d)

In [26]:
precincts.head()

,STATEFP10,COUNTYFP10,VTDST10,GEOID10,VTDI10,NAME10,NAMELSAD10,LSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry
0,42,085,960,42085960,A,SHENANGO TWP VTD WEST,SHENANGO TWP VTD WEST,00,G5240,N,39740056,141805,+41.1564874,-080.4865792,"POLYGON ((-80.51922 41.18555, -80.51905 41.185..."
1,42,039,40,4203940,A,BLOOMING VALLEY Voting District,BLOOMING VALLEY Voting District,00,G5240,N,5034196,57763,+41.6745788,-080.0382865,"POLYGON ((-80.05844 41.68391, -80.05597 41.684..."
2,42,039,10,4203910,A,ATHENS TWP Voting District,ATHENS TWP Voting District,00,G5240,N,73207387,91606,+41.7499682,-079.8467004,"POLYGON ((-79.90979 41.77394, -79.90930 41.773..."
3,42,039,20,4203920,A,BEAVER TWP Voting District,BEAVER TWP Voting District,00,G5240,N,94916802,2059,+41.8099591,-080.4562835,"POLYGON ((-80.51935 41.84956, -80.51544 41.849..."
4,42,039,30,4203930,A,BLOOMFIELD TWP Voting District,BLOOMFIELD TWP Voting District,00,G5240,N,98195373,847815,+41.8113731,-079.8363351,"POLYGON ((-79.90998 41.85069, -79.90995 41.850..."


In [27]:
#Rename columns
precincts = precincts.drop (columns = ['STATEFP10', 'VTDI10', 'NAME10', 'NAMELSAD10', 'LSAD10',
                         'MTFCC10','FUNCSTAT10','ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10', ])
                       # 'cartodb_id', 'other', 'other_perc', 'male', 'male_perce', 'female',
                        # 'female_per', 'total_regi', 'registered', 'id', 'data', 'dem_rep_di'])

precincts.rename(columns = {"GEOID10":"id"}, inplace = True)
#df.rename(columns = {"GEOID10":"canonical"}, inplace = True)
#precincts.rename(columns = {"NEIGHBORS":"neighbors"}, inplace = True)
#df.rename(columns = {"INTPTLON10":"centerY"}, inplace = True)

#Add type columns
precincts.insert(2, 'type', 'ORIGINAL')
precincts.insert(3, 'centerX', None)
precincts.insert(4, 'centerY', None)
precincts.insert(5, 'area', None)
precincts.insert(6, 'neighbors', None)
precincts.insert(7, 'canonical', None)

#precincts = precincts.drop (columns = ['SUM'])
precincts.head()

,COUNTYFP10,VTDST10,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,085,960,ORIGINAL,None,None,None,None,None,42085960,"POLYGON ((-80.51922 41.18555, -80.51905 41.185..."
1,039,40,ORIGINAL,None,None,None,None,None,4203940,"POLYGON ((-80.05844 41.68391, -80.05597 41.684..."
2,039,10,ORIGINAL,None,None,None,None,None,4203910,"POLYGON ((-79.90979 41.77394, -79.90930 41.773..."
3,039,20,ORIGINAL,None,None,None,None,None,4203920,"POLYGON ((-80.51935 41.84956, -80.51544 41.849..."
4,039,30,ORIGINAL,None,None,None,None,None,4203930,"POLYGON ((-79.90998 41.85069, -79.90995 41.850..."


In [29]:
#Canonical name
state = 'PA'
for i, row in precincts.iterrows():   
    id = str(precincts.at[i, 'VTDST10']).zfill(5)
    countyId = precincts.at[i, 'COUNTYFP10']
    #print (countyId)
    canonicalId = state + ' ' + d[countyId] + ' ' + id
    precincts.at[i, 'canonical'] = canonicalId
    precincts.at[i, 'id'] = '42' + str(precincts.at[i, 'COUNTYFP10']) + id
    #print (canonicalId)

precincts.head()

,COUNTYFP10,VTDST10,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,085,960,ORIGINAL,None,None,None,None,PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51905 41.185..."
1,039,40,ORIGINAL,None,None,None,None,PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.05597 41.684..."
2,039,10,ORIGINAL,None,None,None,None,PA Crawford 00010,4203900010,"POLYGON ((-79.90979 41.77394, -79.90930 41.773..."
3,039,20,ORIGINAL,None,None,None,None,PA Crawford 00020,4203900020,"POLYGON ((-80.51935 41.84956, -80.51544 41.849..."
4,039,30,ORIGINAL,None,None,None,None,PA Crawford 00030,4203900030,"POLYGON ((-79.90998 41.85069, -79.90995 41.850..."


In [30]:
#ADD area and centers
for i, row in precincts.iterrows():
    precincts.at[i, 'area'] = row.geometry.area
    bounds = row.geometry.bounds
    centerX = round((bounds[0] + bounds[2]) / 2, 6)
    centerY = round((bounds[1] + bounds[3]) / 2, 6)
    precincts.at[i, 'centerX'] = centerY
    precincts.at[i, 'centerY'] = centerX    

precincts.head()

,COUNTYFP10,VTDST10,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,085,960,ORIGINAL,41.1578,-80.4798,0.00427836,None,PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51905 41.185..."
1,039,40,ORIGINAL,41.6806,-80.0409,0.0005506,None,PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.05597 41.684..."
2,039,10,ORIGINAL,41.7431,-79.841,0.00793395,None,PA Crawford 00010,4203900010,"POLYGON ((-79.90979 41.77394, -79.90930 41.773..."
3,039,20,ORIGINAL,41.8032,-80.4626,0.0102833,None,PA Crawford 00020,4203900020,"POLYGON ((-80.51935 41.84956, -80.51544 41.849..."
4,039,30,ORIGINAL,41.8124,-79.8362,0.0107317,None,PA Crawford 00030,4203900030,"POLYGON ((-79.90998 41.85069, -79.90995 41.850..."


In [32]:
precincts = precincts.drop(columns = ['COUNTYFP10', 'VTDST10'])
precincts.head()

,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,ORIGINAL,41.1578,-80.4798,0.00427836,None,PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51905 41.185..."
1,ORIGINAL,41.6806,-80.0409,0.0005506,None,PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.05597 41.684..."
2,ORIGINAL,41.7431,-79.841,0.00793395,None,PA Crawford 00010,4203900010,"POLYGON ((-79.90979 41.77394, -79.90930 41.773..."
3,ORIGINAL,41.8032,-80.4626,0.0102833,None,PA Crawford 00020,4203900020,"POLYGON ((-80.51935 41.84956, -80.51544 41.849..."
4,ORIGINAL,41.8124,-79.8362,0.0107317,None,PA Crawford 00030,4203900030,"POLYGON ((-79.90998 41.85069, -79.90995 41.850..."


In [ ]:
df.to_file("./PA/PA_full.geojson", driver='GeoJSON')

In [33]:
### GENERATE NEIGHBORS

#file = "./PA/PA.geojson"  
#state_boundary = gp.read_file(file) # open file

precincts["neighbors"] = None  # add NEIGHBORS column

for index, row in precincts.iterrows():
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
        
    neighbors = precincts[~precincts.geometry.disjoint(row.geometry)].id.tolist()
    # remove own name from the list
    neighbors = [ str(name) for name in neighbors if row.id != name ]
    #print(neighbors)
    # add names of neighbors as NEIGHBORS value
    precincts.at[index, 'neighbors'] = ", ".join(neighbors)

precincts.head()


16:51:24 0
16:51:49 1000
16:52:15 2000
16:52:41 3000
16:53:05 4000
16:53:29 5000
16:53:52 6000
16:54:17 7000
16:54:43 8000
16:55:07 9000


,type,centerX,centerY,area,neighbors,canonical,id,geometry
0,ORIGINAL,41.1578,-80.4798,0.00427836,"4207300780, 4207300790, 4208500950, 4208501040...",PA Mercer 00960,4208500960,"POLYGON ((-80.51922 41.18555, -80.51905 41.185..."
1,ORIGINAL,41.6806,-80.0409,0.0005506,"4203900150, 4203900330, 4203900340, 4203900660",PA Crawford 00040,4203900040,"POLYGON ((-80.05844 41.68391, -80.05597 41.684..."
2,ORIGINAL,41.7431,-79.841,0.00793395,"4203900030, 4203900070, 4203900400, 4203900360...",PA Crawford 00010,4203900010,"POLYGON ((-79.90979 41.77394, -79.90930 41.773..."
3,ORIGINAL,41.8032,-80.4626,0.0102833,"4203900090, 4203900420, 4204900040",PA Crawford 00020,4203900020,"POLYGON ((-80.51935 41.84956, -80.51544 41.849..."
4,ORIGINAL,41.8124,-79.8362,0.0107317,"4203900010, 4203900400, 4203900360, 4203900350...",PA Crawford 00030,4203900030,"POLYGON ((-79.90998 41.85069, -79.90995 41.850..."


In [34]:
precincts.to_file("./PA/PA_new.geojson", driver='GeoJSON')

In [ ]:
def create_neighbors_df(neighbors, df):
    neighbors = neighbors.split(',')
    neighbors_df = df[df['id'].isin(neighbors)][['id', 'geometry']]
    return neighbors_df 

In [ ]:
#INTERSECTIONS

file = "./PA/PA.geojson"  
precincts = gp.read_file(file) # open file
precincts = precincts.drop(columns=['type', 'centerX', 'centerY', 'canonical'])
precincts.head()

In [ ]:
AREA_THRESHOLD = 1e-4
intersections = gp.GeoDataFrame(crs = precincts.crs)

for index, row in precincts.iterrows():
    #For progress
    if index % 1000 == 0:
        print (datetime.datetime.now().strftime("%H:%M:%S"), index)
    neighbors = create_neighbors_df(row['neighbors'], precincts)
    #print(neighbors)
    # check if intersection occured
    overlaps = neighbors[neighbors.geometry.overlaps(row.geometry)]['id'].tolist()
    if len(overlaps) > 0:
        # compare the area with threshold
        for y in overlaps:
            temp_area = gp.overlay(precincts.loc[precincts.id == y,], precincts.loc[precincts.id == row.id], how='intersection')
            temp_area = temp_area.loc[temp_area.geometry.area >= AREA_THRESHOLD]
            
            if temp_area.shape[0] > 0:
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections], ignore_index=True),crs = precincts.crs)
                intersections['sorted'] = intersections.apply(lambda y: sorted([y['id_1'],y['id_2']]), axis=1)
                intersections['sorted'] = intersections.sorted.apply(lambda y: ''.join(y))
                intersections = intersections.drop_duplicates('sorted')
                intersections = intersections.reset_index()[['id_1','id_2','geometry']]
                intersections = gp.GeoDataFrame(pd.concat([temp_area, intersections],ignore_index=True),crs=precincts.crs)

In [ ]:
intersections.shape

In [ ]:
#POPULATE DATABASE
mydb = mysql.connector.connect(
    host="mysql3.cs.stonybrook.edu",
    user='mhaczynska',
    passwd="110670160",
    database = 'mhaczynska'    
)

cursor = mydb.cursor()
print (mydb)

In [ ]:
file = "./PA/pa_temp.geojson"
precincts = gp.read_file(file)
precincts.head()
#print(precincts.shape)

In [ ]:
precincts[precincts['id'] == 420031820]

In [ ]:
add_precinct_querry = ("INSERT INTO mhaczynska.precinct "
               "(GEO_ID, NAME, STATE_NAME, CENTER_X, CENTER_Y, NEIGHBORS, PRECINCT_GEOJSON) "
               "VALUES (%s, %s, %s, %s, %s, %s, %s)")

#update_precinct_querry = "UPDATE mhaczynska.precinct SET CENTER_X = %s, CENTER_Y = %s WHERE GEO_ID = %s"

state = 'PA'
#precinct_data = []
for index, row in precincts.iterrows():
    geometry_str = row.geometry.__geo_interface__
    geometry_str = json.dumps(geometry_str)     
    precinct_data = (row['id'], row['canonical'], state, row['centerX'], row['centerY'], row['NEIGHBORS'], geometry_str)
    #precinct_data.append(precinct_row)
    #update_precinct = (row['centerX'], row['centerY'], row['id'])
    cursor.execute(add_precinct_querry, precinct_data)
    mydb.commit()
    

In [ ]:
cursor.close()
mydb.close()